# Demo: Reading Linked Data
* for this exercise you will need to install SPARQLWrapper:
  * __`~/anaconda3/bin/conda install -c conda-forge sparqlwrapper`__
  * Windows: Use Start Menu to open Anaconda Shell, then __`conda install -c conda-forge sparqlwrapper`__

In [1]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [3]:
wds = "https://query.wikidata.org/sparql"

In [4]:
# This is a SPARQL query to send to the SPARQL endpoint defined in the
# previous step. It mixes three vocabularies that each have their own
# definitions but is ultimately a selection from the Wikidata graph.
#
# We're looking for distinct rows of individuals who have an orcid
# (https://orcid.org) and any English descriptions and labels we
# might also have about them. We're matching a pattern in the graph
# for any node that is connected to other nodes with these relationships.
#
# Note that the relationships themselves are often resolvable. To
# understand what wdt:P496 means, expand it into its full URL by
# applying the prefix for wdt and then issue an HTTP request to
# http://www.wikidata.org/prop/direct/P496
#
# For more information on SPARQL, please consult "Learning SPARQL (2nd
# Edition)" by Bob DuCharme.

rq = """
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

select distinct
   ?item
   ?itemLabel
   ?orcid
   ?description
WHERE {
  ?item wdt:P496 ?orcid 
  OPTIONAL { ?item schema:description ?description filter (lang(?description) = "en") }
  SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
  }
} Limit 10000
"""

In [5]:
wikidf = get_sparql_dataframe(wds, rq)

EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'SPARQL-QUERY: queryStr=\nPREFIX bd: <http://www.bigdata.com/rdf#>\nPREFIX wikibase: <http://wikiba.se/ontology#>\nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\n\nselect distinct\n   ?item\n   ?itemLabel\n   ?orcid\n   ?description\nWHERE {\n  ?item wdt:P496 ?orcid \n  OPTIONAL { ?item schema:description ?description filter (lang(?description) = "en") }\n  SERVICE wikibase:label {\n        bd:serviceParam wikibase:language "en" .\n  }\n} Limit 10000\n\njava.util.concurrent.TimeoutException\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:205)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGet(RESTServlet.java:240)\n\tat com.bigd

In [ ]:
# Let's inspect our DataFrame
wikidf.head()

In [ ]:
# Some stats...
wikidf.describe()

In [ ]:
# Top 10 most frequently occurring job titles
wikidf['description'].value_counts()[:10]

In [19]:
# A Jupyter trick to store this DataFrame so we can use it in our exercise
%store wikidf

Stored 'wikidf' (DataFrame)
